In [ ]:
#hw6
#Drashti Snehalkumar Patel

In [1]:
pip install transformers

In [4]:
# load dataset and checking initially
# USED HEADER= NONE to keep all rows intact
#df = pd.read_csv("/content/privacy_prompts(in).csv", header=None)
#prompts = df[0].astype(str).str.strip().tolist()


In [5]:
df.shape

(20, 1)

#Part 1: Generate Responses from Open-Source LLM

In [6]:
from transformers import pipeline
import pandas as pd
import csv

def Part1():
    # Loadin the flan t5 base
    llm = pipeline("text2text-generation", model="google/flan-t5-base", max_length=100)

    # Reading the files
    df = pd.read_csv("/content/privacy_prompts(in).csv", header=None)
    prompts = df[0].astype(str).str.strip().tolist()

    output_rows = []

    for prompt in prompts:
        prompt = prompt.strip()

        #checking files
        agree_input = "Rewrite the following statement to agree: " + prompt
        agree_outputs = llm(agree_input, num_beams=5, num_return_sequences=5)
        agree_text = "GENERATION_FAILED"
        for out in agree_outputs:
            text = out['generated_text'].strip()
            if text != prompt:
                agree_text = text
                break

        disagree_input = "Rewrite the following statement to disagree: " + prompt
        disagree_outputs = llm(disagree_input, num_beams=5, num_return_sequences=5)
        disagree_text = "GENERATION_FAILED"
        for out in disagree_outputs:
            text = out['generated_text'].strip()
            if text != prompt and text != agree_text:
                disagree_text = text
                break

        output_rows.append([agree_text, -1, disagree_text, -1])

    # ensuring row 20 rows r intact
    assert len(output_rows) == 20, f"Expected 20 rows, got {len(output_rows)}"

    # saving csv
    with open("privacy_generated_texts.csv", mode="w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(output_rows)

    print("All 20 rows written to 'privacy_generated_texts.csv'")

#func calling
Part1()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


All 20 rows written to 'privacy_generated_texts.csv'


In [7]:
# printing the generated file
df = pd.read_csv("/content/privacy_generated_texts.csv")
df

,Mass surveillance by the government is justified in order to prevent terrorism and ensure public safety.,-1,Mass surveillance by the government is not necessary to prevent terrorism.,-1.1
0,Companies should be allowed to collect and sel...,-1,User data should be used to improve their serv...,-1
1,Individuals should have the right to demand th...,-1,Individuals should have the right to request t...,-1
2,The right to monitor employees' activities dur...,-1,The right to monitor employees' activities dur...,-1
3,Smart home devices that collect data on users'...,-1,Smart home devices do not collect data on user...,-1
4,Social media platforms should be responsible f...,-1,Social media platforms should be responsible f...,-1
5,"If it leads to medical advancements, sharing m...",-1,It is not acceptable to share medical records ...,-1
6,The use of facial recognition technology in pu...,-1,The use of facial recognition technology in pu...,-1
7,Parents should have the right to monitor their...,-1,Parents should have the right to control their...,-1
8,Governments should be able to break encryption...,-1,Governments should have the ability to break e...,-1
9,Biometric data (like fingerprints and iris sca...,-1,Biometric data can be used to identify a person.,-1


(19, 4)

#Part 2: Human Feedback, Scoring Model

In [9]:
#Train a Scoring Model using Logistic Regression

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# loading our labelled file
df = pd.read_csv("/content/privacy_labeled_texts.csv", header=None)

# Combining both columns of text and labels
texts = df[0].tolist() + df[2].tolist()
labels = df[1].tolist() + df[3].tolist()

# tfidf vectorisation
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

# Training
model = LogisticRegression()
model.fit(X, labels)

#eval
preds = model.predict(X)
accuracy = accuracy_score(labels, preds)

print(f"Training Accuracy: {accuracy}")

Training Accuracy: 0.975


#Part 3: LLM + Scoring Function (LR)





In [10]:

# the model and vectorizer from Part 2 are already loaded for us to work on this part
# model = trained logistic regression
# vectorizer = trained tfidf vectorizer

# Loading the rewritten prompts file
df = pd.read_csv("rewritten_prompts(in).csv", header=None)
prompts = df[0].astype(str).str.strip().tolist()

#FLAN-T5
llm = pipeline("text2text-generation", model="google/flan-t5-base", max_length=100)
guided_output = []

for prompt in prompts:
    # Generate agree and disagree prompts
    agree_input = "Rewrite the following statement to agree: " + prompt
    disagree_input = "Rewrite the following statement to disagree: " + prompt

    # Getting outputs
    agree_text = llm(agree_input, num_beams=5, num_return_sequences=5)[0]['generated_text'].strip()
    disagree_text = llm(disagree_input, num_beams=5, num_return_sequences=5)[0]['generated_text'].strip()

    # rechecking
    if agree_text == prompt:
        agree_text = "AGREE_FAILED"
    if disagree_text == prompt or disagree_text == agree_text:
        disagree_text = "DISAGREE_FAILED"

    agree_score = model.predict_proba(vectorizer.transform([agree_text]))[0][1]
    disagree_score = model.predict_proba(vectorizer.transform([disagree_text]))[0][1]

    # Saving to list
    guided_output.append([agree_text, agree_score, disagree_text, disagree_score])

# Starting csv
df_out = pd.DataFrame(guided_output, columns=["gen_text_1", "prob1", "gen_text_2", "prob2"])
df_out.to_csv("guided_generated_texts.csv", index=False)

print("Output saved to 'guided_generated_texts.csv'")

Device set to use cpu


Output saved to 'guided_generated_texts.csv'
